
https://ar5iv.labs.arxiv.org/html/1201.4145

---



In [ ]:
import networkx as nx
import pandas as pd
import random
from google.colab import drive
import xml.etree.ElementTree as ET

# Mount Google Drive
drive.mount('/content/drive')

# Load GraphML data
graphml_file_path = '/content/drive/MyDrive/ed_proj/network.graphml'

# Create a directed graph
G = nx.DiGraph()

tree = ET.parse(graphml_file_path)
root = tree.getroot()
graphml_ns = {'g': 'http://graphml.graphdrawing.org/xmlns'}

# Extract nodes with attributes
for node in root.findall(".//g:node", namespaces=graphml_ns):
    node_id = node.get('id')
    name = node.find("g:data[@key='v_name']", namespaces=graphml_ns).text
    cluster = int(node.find("g:data[@key='v_cluster']", namespaces=graphml_ns).text)
    G.add_node(node_id, name=name, cluster=cluster)

# Extract edges with weights
for edge in root.findall(".//g:edge", namespaces=graphml_ns):
    source = edge.get('source')
    target = edge.get('target')
    weight = float(edge.find("g:data[@key='e_weight']", namespaces=graphml_ns).text)
    G.add_edge(source, target, weight=weight)


edge_betweenness = nx.edge_betweenness_centrality(G)
threshold = sorted(edge_betweenness.values(), reverse=True)[int(len(edge_betweenness) * 0.05)]  # Top 5% as threshold
eb_ties = [edge for edge, value in edge_betweenness.items() if value >= threshold]
eb_tie_nodes = set()
for edge in eb_ties:
    eb_tie_nodes.update(edge)

# Identify Pro-ED nodes and calculate centralities
pro_ed_nodes = [node for node, attr in G.nodes(data=True) if attr['cluster'] == 0]
degree_centrality = nx.in_degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
pagerank = nx.pagerank(G)

# Number of starting nodes for each method
num_start_nodes = 100

# Select the top nodes for each centrality measure
top_degree = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:num_start_nodes]
top_betweenness = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:num_start_nodes]
top_closeness = sorted(closeness_centrality, key=closeness_centrality.get, reverse=True)[:num_start_nodes]
top_eigenvector = sorted(eigenvector_centrality, key=eigenvector_centrality.get, reverse=True)[:num_start_nodes]
top_pagerank = sorted(pagerank, key=pagerank.get, reverse=True)[:num_start_nodes]


eb_tie_nodes = list(eb_tie_nodes)[:num_start_nodes]



# Merge Degree and Weak Tie nodes (maintaining equal number)
starting_nodes_with_edge_betweeness = set(top_degree[:num_start_nodes//2])  # Half from degree centrality
starting_nodes_with_edge_betweeness.update(eb_tie_nodes[:num_start_nodes//2])  # Half from weak ties

# Merge PageRank and Betweenness nodes (maintaining equal number)
starting_nodes_with_page_rank_and_betweenness = set(top_pagerank[:num_start_nodes//2])  # Half from PageRank
starting_nodes_with_page_rank_and_betweenness.update(top_betweenness[:num_start_nodes//2])  # Half from betweenness

# Information Diffusion Simulation
def simulate_diffusion_pro_ed(G, start_nodes, steps=5):
    reached_nodes = set(start_nodes)  # Nodes that have received information
    frontier = set(start_nodes)      # Nodes that will spread information in the next step

    for _ in range(steps):
        if not frontier:  # Stop if there are no more nodes to spread to
            break
        next_frontier = set()  # Nodes that will spread information in the next step
        for node in frontier:
            for neighbor in G.predecessors(node):
                if neighbor in pro_ed_nodes and neighbor not in reached_nodes:
                    if random.random() < 0.50:  # 50% chance to spread information
                        reached_nodes.add(neighbor)
                        next_frontier.add(neighbor)
        frontier = next_frontier  # Update the frontier

    return len(frontier), reached_nodes  # Return the number of steps and the set of reached nodes



# Run simulations with equal number of starting nodes for all methods
results = []
for method, nodes in [('Degree', top_degree), ('Betweenness', top_betweenness),
                      ('Closeness', top_closeness), ('Eigenvector', top_eigenvector),
                      ('PageRank', top_pagerank),
                      ('Degree with edge betweeness', list(starting_nodes_with_edge_betweeness)), ('nodes_with_page_rank_and_betweennes', list(starting_nodes_with_page_rank_and_betweenness))]:
    steps_taken, reached_nodes = simulate_diffusion_pro_ed(G, nodes, steps=5)
    results.append({'Method': method, 'Steps': steps_taken, 'Nodes Reached': len(reached_nodes), 'Average per step': len(reached_nodes)/(steps_taken+1)})

# Convert results to a DataFrame for comparison
results_df = pd.DataFrame(results)
print(results_df)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                Method  Steps  Nodes Reached  Average per step
0                               Degree      2           1905        635.000000
1                          Betweenness      0            634        634.000000
2                            Closeness      3           1599        399.750000
3                          Eigenvector      0            116        116.000000
4                             PageRank      1           1787        893.500000
5          Degree with edge betweeness      0           1819       1819.000000
6  nodes_with_page_rank_and_betweennes      2           1676        558.666667


In [ ]:
# Number of starting nodes for each method
num_start_nodes = 60

# Select the top nodes for each centrality measure
top_degree = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:num_start_nodes]
top_betweenness = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:num_start_nodes]
top_closeness = sorted(closeness_centrality, key=closeness_centrality.get, reverse=True)[:num_start_nodes]
top_eigenvector = sorted(eigenvector_centrality, key=eigenvector_centrality.get, reverse=True)[:num_start_nodes]
top_pagerank = sorted(pagerank, key=pagerank.get, reverse=True)[:num_start_nodes]


eb_tie_nodes = list(eb_tie_nodes)[:num_start_nodes]



# Merge Degree and Weak Tie nodes (maintaining equal number)
starting_nodes_with_edge_betweeness = set(top_degree[:num_start_nodes//2])  # Half from degree centrality
starting_nodes_with_edge_betweeness.update(eb_tie_nodes[:num_start_nodes//2])  # Half from weak ties

# Merge PageRank and Betweenness nodes (maintaining equal number)
starting_nodes_with_page_rank_and_betweenness = set(top_pagerank[:num_start_nodes//2])  # Half from PageRank
starting_nodes_with_page_rank_and_betweenness.update(top_betweenness[:num_start_nodes//2])  # Half from betweenness

# Information Diffusion Simulation
def simulate_diffusion_pro_ed(G, start_nodes, steps=5):
    reached_nodes = set(start_nodes)  # Nodes that have received information
    frontier = set(start_nodes)      # Nodes that will spread information in the next step

    for _ in range(steps):
        if not frontier:  # Stop if there are no more nodes to spread to
            break
        next_frontier = set()  # Nodes that will spread information in the next step
        for node in frontier:
            for neighbor in G.predecessors(node):
                if neighbor in pro_ed_nodes and neighbor not in reached_nodes:
                    if random.random() < 0.50:  # 50% chance to spread information
                        reached_nodes.add(neighbor)
                        next_frontier.add(neighbor)
        frontier = next_frontier  # Update the frontier

    return len(frontier), reached_nodes  # Return the number of steps and the set of reached nodes



# Run simulations with equal number of starting nodes for all methods
results = []
for method, nodes in [('Degree', top_degree), ('Betweenness', top_betweenness),
                      ('Closeness', top_closeness), ('Eigenvector', top_eigenvector),
                      ('PageRank', top_pagerank),
                      ('Degree with edge betweeness', list(starting_nodes_with_edge_betweeness)), ('nodes_with_page_rank_and_betweennes', list(starting_nodes_with_page_rank_and_betweenness))]:
    steps_taken, reached_nodes = simulate_diffusion_pro_ed(G, nodes, steps=5)
    results.append({'Method': method, 'Steps': steps_taken, 'Nodes Reached': len(reached_nodes), 'Average per step': len(reached_nodes)/(steps_taken+1)})

# Convert results to a DataFrame for comparison
results_df = pd.DataFrame(results)
print(results_df)


                                Method  Steps  Nodes Reached  Average per step
0                               Degree      2           1806        602.000000
1                          Betweenness      5            557         92.833333
2                            Closeness     16           1419         83.470588
3                          Eigenvector      0             73         73.000000
4                             PageRank      2           1757        585.666667
5          Degree with edge betweeness      4           1623        324.600000
6  nodes_with_page_rank_and_betweennes     11           1552        129.333333


In [ ]:
# Number of starting nodes for each method
num_start_nodes = 50

# Select the top nodes for each centrality measure
top_degree = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:num_start_nodes]
top_betweenness = sorted(betweenness_centrality, key=betweenness_centrality.get, reverse=True)[:num_start_nodes]
top_closeness = sorted(closeness_centrality, key=closeness_centrality.get, reverse=True)[:num_start_nodes]
top_eigenvector = sorted(eigenvector_centrality, key=eigenvector_centrality.get, reverse=True)[:num_start_nodes]
top_pagerank = sorted(pagerank, key=pagerank.get, reverse=True)[:num_start_nodes]


eb_tie_nodes = list(eb_tie_nodes)[:num_start_nodes]



# Merge Degree and Weak Tie nodes (maintaining equal number)
starting_nodes_with_edge_betweeness = set(top_degree[:num_start_nodes//2])  # Half from degree centrality
starting_nodes_with_edge_betweeness.update(eb_tie_nodes[:num_start_nodes//2])  # Half from weak ties

# Merge PageRank and Betweenness nodes (maintaining equal number)
starting_nodes_with_page_rank_and_betweenness = set(top_pagerank[:num_start_nodes//2])  # Half from PageRank
starting_nodes_with_page_rank_and_betweenness.update(top_betweenness[:num_start_nodes//2])  # Half from betweenness

# Information Diffusion Simulation
def simulate_diffusion_pro_ed(G, start_nodes, steps=5):
    reached_nodes = set(start_nodes)  # Nodes that have received information
    frontier = set(start_nodes)      # Nodes that will spread information in the next step

    for _ in range(steps):
        if not frontier:  # Stop if there are no more nodes to spread to
            break
        next_frontier = set()  # Nodes that will spread information in the next step
        for node in frontier:
            for neighbor in G.predecessors(node):
                if neighbor in pro_ed_nodes and neighbor not in reached_nodes:
                    if random.random() < 0.50:  # 50% chance to spread information
                        reached_nodes.add(neighbor)
                        next_frontier.add(neighbor)
        frontier = next_frontier  # Update the frontier

    return len(frontier), reached_nodes  # Return the number of steps and the set of reached nodes



# Run simulations with equal number of starting nodes for all methods
results = []
for method, nodes in [('Degree', top_degree), ('Betweenness', top_betweenness),
                      ('Closeness', top_closeness), ('Eigenvector', top_eigenvector),
                      ('PageRank', top_pagerank),
                      ('Degree with edge betweeness', list(starting_nodes_with_edge_betweeness)), ('nodes_with_page_rank_and_betweennes', list(starting_nodes_with_page_rank_and_betweenness))]:
    steps_taken, reached_nodes = simulate_diffusion_pro_ed(G, nodes, steps=5)
    results.append({'Method': method, 'Steps': steps_taken, 'Nodes Reached': len(reached_nodes), 'Average per step': len(reached_nodes)/(steps_taken+1)})

# Convert results to a DataFrame for comparison
results_df = pd.DataFrame(results)
print(results_df)

#

                                Method  Steps  Nodes Reached  Average per step
0                               Degree      2           1755        585.000000
1                          Betweenness      4            546        109.200000
2                            Closeness      4           1332        266.400000
3                          Eigenvector      0             64         64.000000
4                             PageRank     10           1616        146.909091
5          Degree with edge betweeness      1           1504        752.000000
6  nodes_with_page_rank_and_betweennes      8           1365        151.666667
